In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re, os, sys

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
import word_utils

Using TensorFlow backend.


In [8]:
df = word_utils.load_dictionary()
df = df.rename(columns={'word':'Surname'})

df_names = pd.read_csv('../data/raw/names/lastnames.csv')
df_names['Surname'] = df_names['Surname'].str.replace("'","")
df_names['Surname'] = df_names['Surname'].str.replace(".","")
df_names = pd.merge(df_names, df, on='Surname', how='left')

print('Count of names:', len(df_names))
not_matched = df_names[df_names['phonetics'] != df_names['phonetics']]
print('Un-matched names:', len(not_matched))

Size of word dictionary: 117414
Count of names: 6453
Un-matched names: 740


In [9]:
not_matched = word_utils.load_and_predict(not_matched,'Surname')
not_matched = not_matched.drop('phonetics', axis=1)
not_matched.head()

Loaded model from disk


/Users/chrisdoughty/Desktop/Git/Analysis_BabyNames/src/word_utils.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['phonetic_predict'] = master


,Surname,phonetic_predict
458,KAUR,K AO1 R
498,MCINALLY,M AH0 K IH1 N AH0 IY0
514,CAIRNEY,K EH1 R N IY0
596,NEWLANDS,N UW1 L AH0 N D Z
607,MCLAUCHLAN,M AH0 K L AE1 K K AH0 N


In [10]:
df_names = pd.merge(df_names, not_matched, on='Surname', how='left')

# First name phonetic source using columns
df_names['phonetics_source'] = np.where(df_names['phonetic_predict'] != df_names['phonetic_predict'],
                                         'dict', 'pred')

# Standardise the phonetic column
df_names['phonetics'] = np.where(df_names['phonetics_source'] == 'dict', 
                                 df_names['phonetics'], df_names['phonetic_predict'])

# Remove the unused phonetic prediction column from our DataFrame
df_names = df_names.drop('phonetic_predict', axis=1)

# Create a field without spaces
df_names['phonetics_space'] = df_names['phonetics'].str.replace(' ','')

# Add the last sound of the string
df_names['final_sound'] = df_names['phonetics'].str.split(' ').str[-1]

df_names.to_csv('../data/interim/lastnames.csv', index=False)

In [11]:
df_first = pd.read_csv('../data/raw/names/firstnames.csv')

df_first['Count'] = df_first['Count'].str.replace(',','')
df_first['Count'] = df_first['Count'].astype(int)
df_first = df_first[df_first['Count'] >= 3]
df_first['Name'] = df_first['Name'].str.upper()
df_first['Name'] = df_first['Name'].str.replace(' ','')
df_first['Name'] = df_first['Name'].str.replace("'","")
df_first['Name'] = df_first['Name'].str.replace(".","")
df_first = df_first[['Name']]
print('Original DataFrame:', len(df_first))

df_first.drop_duplicates(inplace=True)
print('DeDuped DataFrame:', len(df_first))

df = word_utils.load_dictionary()
df = df.rename(columns={'word':'Name'})
df_first = pd.merge(df_first, df, on='Name', how='left')

print('Count of names:', len(df_first))
not_matched_first = df_first[df_first['phonetics'] != df_first['phonetics']]
print('Un-matched names:', len(not_matched_first))

Original DataFrame: 250078
DeDuped DataFrame: 31697
Size of word dictionary: 117414
Count of names: 31697
Un-matched names: 25036


In [12]:
not_matched_first = word_utils.load_and_predict(not_matched_first,'Name')
not_matched_first.head()

Loaded model from disk


/Users/chrisdoughty/Desktop/Git/Analysis_BabyNames/src/word_utils.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['phonetic_predict'] = master


,Name,phonetics,phonetic_predict
15,EVIE,NaN,EH1 V IY0
20,EILIDH,NaN,AY1 L IH0 D
53,LEXI,NaN,L EH1 K S IY0
61,AYLA,NaN,EY1 L AH0
82,MYA,NaN,M IY1 AH0


In [13]:
not_matched_first = not_matched_first.drop('phonetics', axis=1)
df_first = pd.merge(df_first, not_matched_first, on='Name', how='left')

# First name phonetic source using columns
df_first['phonetics_source'] = np.where(df_first['phonetic_predict'] != df_first['phonetic_predict'],
                                         'dict', 'pred')

# Standardise the phonetic column
df_first['phonetics'] = np.where(df_first['phonetics_source'] == 'dict', 
                                 df_first['phonetics'], df_first['phonetic_predict'])

# Remove the unused phonetic prediction column from our DataFrame
df_first = df_first.drop('phonetic_predict', axis=1)

# Create a field without spaces
df_first['phonetics_space'] = df_first['phonetics'].str.replace(' ','')

# Add the last sound of the string
df_first['final_sound'] = df_first['phonetics'].str.split(' ').str[-1]

df_first['single'] = df_first['Name'].str[0]
df_first['double'] = df_first['Name'].str[0:2]

df_first.to_csv('../data/interim/frstnames.csv', index=False)